In [1]:
%pip install --upgrade langchain langchain-ollama langchain-community chromadb
%pip install fastapi nest-asyncio pyngrok uvicorn line-bot-sdk python-dotenv
%pip install fastapi uvicorn python-multipart requests

  Using cached jsonschema-4.23.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached starlette-0.45.3-py3-none-any.whl.metadata (6.3 kB)
  Using cached jsonschema_specifications-2024.10.1-py3-none-any.whl.metadata (3.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 22.8 MB/s eta 0:00:00a 0:00:01
Using cached jsonschema-4.23.0-py3-none-any.whl (88 kB)
Using cached jsonschema_specifications-2024.10.1-py3-none-any.whl (18 kB)
Using cached starlette-0.45.3-py3-none-any.whl (71 kB)
  Attempting uninstall: starlette
    Found existing installation: starlette 0.46.1
    Uninstalling starlette-0.46.1:
      Successfully uninstalled starlette-0.46.1
  Attempting uninstall: fastapi
    Found existing installation: fastapi 0.115.11
    Uninstalling fastapi-0.115.11:
      Successfully uninstalled fastapi-0.115.11
  Attempting uninsta

In [2]:
from langchain_ollama import OllamaLLM, OllamaEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

import nest_asyncio
from pyngrok import ngrok
import uvicorn
from fastapi import FastAPI, Request, HTTPException
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage

หั่นคำใส่ LLM

In [3]:
def load_documents(directory):
    documents = []
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            file_path = os.path.join(directory, filename)
            loader = TextLoader(file_path, encoding='utf-8')
            documents.extend(loader.load())
    return documents

docs = load_documents("./")

In [4]:
# llm = OllamaLLM(
#     model="scb10x/llama3.2-typhoon2-1b-instruct",
#     temperature=0.3,
# )

# embeddings = OllamaEmbeddings(
#     model="scb10x/llama3.2-typhoon2-1b-instruct"
# )

# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=800,
#     chunk_overlap=200,
#     separators=["\n\n", "\n", " ", ""]
# )

# split_docs = text_splitter.split_documents(docs)

# vectorstore = Chroma.from_documents(
#     split_docs,
#     embeddings,
#     collection_name="my_collection"  # ไม่ต้องใส่ persist_directory
# )
# retriever = vectorstore.as_retriever()

In [5]:
llm = OllamaLLM(
    model="llama3.2",
    temperature=0.3,
)

embeddings = OllamaEmbeddings(
    model="llama3.2"
)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=200,
    separators=["\n\n", "\n", " ", ""]
)

split_docs = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(split_docs, embeddings)
retriever = vectorstore.as_retriever()

# https://p6wxt3k9-8000.asse.devtunnels.ms/

In [6]:
prompt_template = """
คุณคือผู้ให้คำปรึกษาที่ชื่อว่า "น้องไตน้อย" ผู้ช่วยให้คำปรึกษาที่มีความรู้ความเข้าใจลึกซึ้งในเรื่องการดูแลสุขภาพของผู้ป่วยโรคไตโดยเน้นไปทางด้านอาหารการกิน การคำนวณสารอาหาร คุณสามารถตอบคำถามเกี่ยวกับโรคไต การดูแลสุขภาพได้ 
ตอบคำถามต่อไปนี้โดยยึดตามข้อมูลที่ให้มาเท่านั้น และควรแน่ใจว่าคำตอบถูกต้องตามข้อมูลที่มีอยู่ โดยจะมีข้อมูลผู้ใช้แนบมา โดยความหมายตัวแปรต่าง ๆ จะอยู่ในข้อมูลที่ให้ไป โดยแต่ละตัวแปรจะมีความหมายของมันซึ่งจะเป็นข้อมูลของผู้ใช้ 
ถ้าไม่มั่นใจว่ามีข้อมูลก็ควรบอกว่า "ขออภัยไม่สามารถตอบคำถามนี้ได้ คำถามของคุณจะถูกบันทึกไว้เพื่อปรับปรุงคุณภาพของเรา"
ใส่อีโมจิต่าง ๆ ในแต่ละคำตอบด้วย เพื่อความ Friendly มากขึ้น เช่น "✨", "👋"  
และตอบด้วยความชัดเจน กระชับ ในภาษาไทย ลงท้ายด้วยคำว่า "ครับ" เพื่อให้สุภาพ และแทนตัวเองด้วย "น้องไตน้อย" ให้ตอบคำถามให้กระชับ สั้น ๆ และเข้าใจง่ายมากที่สุด


บริบท: {context}

คำถาม: {question}


คำตอบ:
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

In [7]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

def ask_question(data):
    result = qa_chain.invoke({"query": data})
    return result["result"]

In [8]:
question = "โรคไตผมต้องระวังอะไรบ้าง"

data = f"{question}"
answer = ask_question(data)
print(f"คำถาม: {question}")
print(f"คำตอบ: {answer}")

คำถาม: โรคไตผมต้องระวังอะไรบ้าง
คำตอบ: 🤔 "น้องไตน้อย" ต้องการให้คุณรู้ว่าโรคไตผมต้องระวังเรื่องของการดื่มน้ำมากเกินไป 🚽 หากคุณมีอาการบวมน้ำหรือภาวะหัวใจล้มเหลว ควรปรับลดปริมาณน้ำที่ดื่มลง 👍

นอกจากนี้ โรคไตผมต้องระวังเรื่องของอาหารเค็ม 🤢 และควรหลีกเลี่ยงการบริโภคอาหารที่มีโซเดียมสูง เช่น น้ำผลไม้และเครื่องดื่มแอลกอฮอล์ 🍹

นอกจากนี้ ควรตรวจสอบระดับโพแทสเซียมในเลือดอย่างต่อเนื่อง และควบคุมปริมาณการบริโภคฟอสฟอรัสให้เหมาะสม 👀


FastAPI เรียกใช้กรณีรูป

In [9]:
from fastapi import FastAPI, File, UploadFile
from dotenv import load_dotenv
import requests
import shutil


load_dotenv()
app = FastAPI()



AI_API_URL = f"https://ai-detect-1025044834972.us-central1.run.app/detect-foods/"

@app.post("/upload/")
async def upload_image(file: UploadFile = File(...)):

    file_location = f"temp/{file.filename}"
    with open(file_location, "wb") as buffer:
        shutil.copyfileobj(file.file, buffer)


    with open(file_location, "rb") as image_file:
        response = requests.post(AI_API_URL, files={"file": image_file})

    return response.json()


เรียก Line 

In [10]:
from dotenv import load_dotenv
from pyngrok import ngrok
import os
import json

load_dotenv()
app = FastAPI()

line_bot_api = LineBotApi(os.getenv('LINE_CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(os.getenv('LINE_CHANNEL_SECRET'))
ngrok.set_auth_token(os.getenv('NGROK_AUTH_TOKEN'))

/var/folders/0h/jv4lpygs6nx8s7rn4yvxrwwm0000gn/T/ipykernel_78302/2831993446.py:9: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi(os.getenv('LINE_CHANNEL_ACCESS_TOKEN'))
/var/folders/0h/jv4lpygs6nx8s7rn4yvxrwwm0000gn/T/ipykernel_78302/2831993446.py:10: LineBotSdkDeprecatedIn30: Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  handler = WebhookHandler(os.getenv('LINE_CHANNEL_SECRET'))


In [ ]:
import requests
import os
from linebot.models import MessageEvent, ImageMessage
from fastapi import FastAPI
import json
from linebot.models import FlexSendMessage, BubbleContainer, CarouselContainer, TextComponent, ButtonComponent, URIAction, PostbackEvent, PostbackAction

def generate_select_recipe_flex(recipes):
    bubbles = []

    for recipe in recipes[:12]:
        bubble = {
            "type": "bubble",
            "size": "micro",
            "hero": {
                "type": "image",
                "url": recipe.get("image_url", ["https://via.placeholder.com/300"])[0],
                "size": "full",
                "aspectMode": "cover",
                "aspectRatio": "1:1"
            },
            "body": {
                "type": "box",
                "layout": "vertical",
                "spacing": "sm",
                "contents": [
                    {
                        "type": "text",
                        "text": recipe["recipes_name"],
                        "wrap": True,
                        "weight": "bold",
                        "size": "md"
                    }
                ]
            },
            "footer": {
                "type": "box",
                "layout": "vertical",
                "spacing": "sm",
                "contents": [
                    {
                        "type": "button",
                        "style": "primary",
                        "color": "#FFA500",
                        "action": {
                            "type": "postback",
                            "label": "เลือกเมนูนี้",
                            "data": f"SELECT_RECIPE_{recipe['recipes_id']}"
                        }
                    }
                ]
            }
        }
        bubbles.append(bubble)

    flex_message = FlexSendMessage(
        alt_text="เลือกเมนูอาหารที่คุณต้องการ",
        contents={
            "type": "carousel",
            "contents": bubbles
        }
    )

    return flex_message
def generate_flex_message(food_detail):
    return FlexSendMessage(
        alt_text="ข้อมูลอาหาร",
        contents={
            "type": "bubble",
            "hero": {
                "type": "image",
                "url": food_detail['image_url'][0],
                "size": "full",
                "aspectRatio": "20:13",
                "aspectMode": "cover"
            },
            "body": {
                "type": "box",
                "layout": "vertical",
                "spacing": "md",
                "contents": [
                    {
                        "type": "text",
                        "text": food_detail['recipe_name'],
                        "weight": "bold",
                        "size": "xl",
                        "wrap": True
                    },
                    {
                        "type": "text",
                        "text": f"แคลอรี่: {food_detail['calories']} kcal | โปรตีน: {food_detail['protein']}g | ไขมัน: {food_detail['fat']}g",
                        "size": "sm",
                        "color": "#666666",
                        "wrap": True
                    }
                ]
            }
        }
    )


app = FastAPI()


# โหลดกำลังพิมพ์
def start_loading_animation(user_id):
    url = "https://api.line.me/v2/bot/chat/loading/start"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {(os.getenv('LINE_CHANNEL_ACCESS_TOKEN'))}"
    }
    data = {"chatId": user_id}
    
    response = requests.post(url, headers=headers, json=data, timeout=10)



# โหลดข้อมูลจาก Rust Sqlx
def loading_userInformation(user_id):
    rust_server_url = "https://backend-billowing-waterfall-4640.fly.dev/chatbot/"
    try:
        response = requests.get(f"{rust_server_url}{user_id}",timeout = 7)
        response.raise_for_status()  
        global user_information
        user_information = response.text
        return response.json()
    
    except requests.exceptions.Timeout:
        print(f"การเชื่อมต่อกับ Rust server หมดเวลา")
        return None
    
    except requests.exceptions.RequestException as e:
        print(f"เกิดข้อผิดพลาดในการเชื่อมต่อ sqlx: {e}")
        return None 


@app.post("/callback")
async def callback(request: Request):
    signature = request.headers['X-Line-Signature']
    body = await request.body()

    try:
        handler.handle(body.decode("utf-8"), signature)
    except InvalidSignatureError:
        raise HTTPException(status_code=400, detail="Invalid signature")

    return 'OK'



# AI Zone

# แก้ให้ query ผ่าน API Food
def get_name(FoodID):
    rust_server_url = "https://backend-billowing-waterfall-4640.fly.dev/food_details/"
    try:
        response = requests.get(f"{rust_server_url}{FoodID}",timeout = 7)
        response.raise_for_status()  
        global user_information
        user_information = response.text
        return response.json()

    except requests.exceptions.Timeout:
        print(f"การเชื่อมต่อกับ Rust server หมดเวลา")
        return None
    
    except requests.exceptions.RequestException as e:
        print(f"เกิดข้อผิดพลาดในการเชื่อมต่อ sqlx: {e}")
        return None 

@handler.add(PostbackEvent)
def handle_postback(event):
    
    data = event.postback.data

    if data.startswith("SELECT_RECIPE_"):
        user_id = event.source.user_id  
        FoodID = int(data.replace("SELECT_RECIPE_", ""))
        start_loading_animation(user_id)  
        food_detail = get_name(FoodID)
        flex_msg = generate_flex_message(food_detail)

        line_bot_api.reply_message(
            event.reply_token,
            flex_msg
        )

# ดึงข้อมูลจาก AI หลังอัพรูป
@handler.add(MessageEvent, message=ImageMessage)
def handle_image_message(event):
    message_id = event.message.id

    user_id = event.source.user_id  
    start_loading_animation(user_id)  

    image_content = line_bot_api.get_message_content(message_id)

    image_path = f"temp_{message_id}.jpg"
    with open(image_path, "wb") as f:
        for chunk in image_content.iter_content():
            f.write(chunk)

    # ส่งไปยัง Roboflow 
    try:
        with open(image_path, "rb") as img:
            response = requests.post(AI_API_URL, files={"file": img}, timeout = 20)

        result = response.json()
        recipes = result.get("recipes", [])
        print("response", recipes)
        
        if recipes:
            Flex_msg = generate_select_recipe_flex(recipes)

            line_bot_api.reply_message(
                event.reply_token,
                [
                    TextMessage(text="กรุณาเลือกเนื้อสัตว์ของอาหารของคุณ"),
                    Flex_msg
                ]
            )
            # print("เจอ", FoodID)
        else :
            line_bot_api.reply_message (
                event.reply_token,
                TextMessage(text="ขอโทษด้วยเราไม่สามารถตรวจจับรูปที่ไม่ใช่อาหารได้")
            )


        

    except Exception as e:
        print(f"err : {e}")

    finally:
        if os.path.exists(image_path):
            os.remove(image_path)  

    




    
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id  
    print(f"User ID: {user_id}")
    start_loading_animation(user_id)  
    loading_userInformation(user_id)
    
    # ดึง rust มาทำงาน
    text = event.message.text


    try:
        data = f"{text} {user_information}"
        if user_information == None:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text="ขออภัย Server เกิดข้อผิดพลาด กรุณาลองใหม่อีกครั้ง")
            )
        else:
            reply_text = ask_question(data)
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text=reply_text)
            )
        

    except Exception as e:
        print(f"Error in handle_message: {e}")
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text="เกิดข้อผิดพลาด กรุณาลองใหม่อีกครั้ง")
        )


def ask_question_with_timeout(data, timeout=20):
    try:
        # ถ้า ask_question เป็น API call หรือกระบวนการที่ใช้เวลานาน
        # ใช้ threading หรือ asyncio เพื่อกำหนด timeout
        from concurrent.futures import ThreadPoolExecutor
        from concurrent.futures import TimeoutError
        
        with ThreadPoolExecutor() as executor:
            future = executor.submit(ask_question, data)
            try:
                return future.result(timeout=timeout)
            except TimeoutError:
                return "ขออภัย การประมวลผลใช้เวลานานเกินไป กรุณาลองใหม่อีกครั้ง"
    except Exception as e:
        print(f"Error in ask_question: {e}")
        return "เกิดข้อผิดพลาดในการประมวลผล กรุณาลองใหม่อีกครั้ง"
ngrok_tunnel = ngrok.connect(8082)
print('Webhook URL:', ngrok_tunnel.public_url + '/callback')

nest_asyncio.apply()

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8082)



Webhook URL: https://4ee8-220-70-170-23.ngrok-free.app/callback


INFO:     Started server process [78302]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8082 (Press CTRL+C to quit)
/var/folders/0h/jv4lpygs6nx8s7rn4yvxrwwm0000gn/T/ipykernel_78302/2981899146.py:196: LineBotSdkDeprecatedIn30: Call to deprecated method get_message_content. (Use 'from linebot.v3.messaging import MessagingApiBlob' and 'MessagingApiBlob(...).get_message_content(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  image_content = line_bot_api.get_message_content(message_id)


response [{'recipes_id': 1, 'recipes_name': 'ข้าวผัดไข่ขาว'}, {'recipes_id': 7, 'recipes_name': 'ข้าวผัดเบญจรงค์'}, {'recipes_id': 23, 'recipes_name': 'ข้าวผัดหมู'}, {'recipes_id': 24, 'recipes_name': 'ข้าวผัดไก่'}, {'recipes_id': 25, 'recipes_name': 'ข้าวผัดกุ้ง'}, {'recipes_id': 26, 'recipes_name': 'ข้าวผัดปู'}, {'recipes_id': 27, 'recipes_name': 'ข้าวผัดปลาหมึก'}, {'recipes_id': 28, 'recipes_name': 'ข้าวผัดมันกุ้ง'}, {'recipes_id': 29, 'recipes_name': 'ข้าวผัดแหนม'}, {'recipes_id': 30, 'recipes_name': 'ข้าวผัดแฮม'}, {'recipes_id': 31, 'recipes_name': 'ข้าวผัดไส้กรอก'}, {'recipes_id': 32, 'recipes_name': 'ข้าวผัดไข่'}, {'recipes_id': 33, 'recipes_name': 'ข้าวผัดอเมริกัน'}, {'recipes_id': 34, 'recipes_name': 'ข้าวผัดหมูกรอบ'}, {'recipes_id': 35, 'recipes_name': 'ข้าวผัดเบคอน'}, {'recipes_id': 36, 'recipes_name': 'ข้าวผัดกุนเชียง'}, {'recipes_id': 37, 'recipes_name': 'ข้าวผัดแซลมอน'}, {'recipes_id': 38, 'recipes_name': 'ข้าวผัดหอยแมลงภู่'}, {'recipes_id': 39, 'recipes_name': 'ข้าวผัดเน

/var/folders/0h/jv4lpygs6nx8s7rn4yvxrwwm0000gn/T/ipykernel_78302/2981899146.py:218: LineBotSdkDeprecatedIn30: Call to deprecated class TextMessage. (Use 'from linebot.v3.webhooks import TextMessageContent' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  TextMessage(text="กรุณาเลือกเนื้อสัตว์ของอาหารของคุณ"),
/var/folders/0h/jv4lpygs6nx8s7rn4yvxrwwm0000gn/T/ipykernel_78302/2981899146.py:215: LineBotSdkDeprecatedIn30: Call to deprecated method reply_message. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).reply_message(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.reply_message(


INFO:     147.92.150.196:0 - "POST /callback HTTP/1.1" 200 OK


/var/folders/0h/jv4lpygs6nx8s7rn4yvxrwwm0000gn/T/ipykernel_78302/2981899146.py:183: LineBotSdkDeprecatedIn30: Call to deprecated method reply_message. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).reply_message(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.reply_message(


INFO:     147.92.150.196:0 - "POST /callback HTTP/1.1" 200 OK
